In [1]:
df = spark.sql("SELECT * FROM bing_LH.tbl_latest_ai_news LIMIT 1000")
display(df)

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2d34069d-c4c2-475b-815d-68fac7685a5a)

In [2]:
#import required packages
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 4, Finished, Available, Finished)

In [3]:
#initialise the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description") ## set the column we want to perform sentiments on
        .setKind("SentimentAnalysis") ## specifying the sentiment analysis model to be performed.
        .setOutputCol("response")
        .setErrorCol("error")) 

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 5, Finished, Available, Finished)

In [4]:
#Apply the model to our dataframe
result = model.transform(df)

display(result.limit(10))

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, df8f7962-6fd8-47d9-a785-e89786784a68)

In [5]:
#To get the Sentiment Column from the response column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))
display(sentiment_df.limit(100))

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d26b23b0-26de-4a94-b111-507b9ff590ce)

In [6]:
#Droping the error and response columns

sentiment_df_final = sentiment_df.drop("error","response")
display(sentiment_df_final.limit(10))

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 53276df5-0a01-4d4f-8d75-0004403aa282)

##### **Implementing the Type 1 incremental Loading of the Sentiment Data**

In [7]:
# Adopting TYPE 1 SCD incremental loading for our data.

'''In a Type 1 SCD the new data overwrites the existing data without duplicate. Thus the existing data
 is lost as it is not stored anywhere else. This is typically used when there is no need to keep 
 a history of the data.'''

from pyspark.sql.utils import AnalysisException

#Exception Handling
try:

    table_name = "bing_LH.tbl_sentiment_analysis"
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print ("Table Already Exist")

    sentiment_df.createOrReplaceTempView("vw_sentiment")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment source_view

                    ON source_view.link = target_table.link

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.image <> target_table.image OR
                    source_view.link <> target_table.link OR
                    source_view.datePublished <> target_table.datePublished OR
                    source_view.provider <> target_table.provider OR
                    source_view.published_date <> target_table.published_date OR
                    source_view.published_time<> target_table.published_time
                    
                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT * 

                """)
     

StatementMeta(, e8caaf44-824b-4d4f-8cdc-53fd7d19bd37, 9, Finished, Available, Finished)